# NanoHUB Submit Python Tutorial

This notebook is a practical guide for `nanohubsubmit`, including interactive controls with `ipywidgets<8`.

Covered topics:
- Create a client using the default config (`/etc/submit/submit-client.conf`)
- Run doctor/health checks
- Query tools, venues, managers
- Explore server capabilities
- Use raw protocol-compatible queries
- Validate submit requests before sending
- Submit local and remote jobs
- Query status and kill jobs
- Inspect local sessions
- Handle errors/timeouts cleanly
- Interactive widget-based runner


## 1) Install (Notebook Extras)

Run once in your environment if needed:

```bash
python3 -m pip install -e ".[notebook]"
```

If you use classic Jupyter Notebook with `ipywidgets<8`, you may also need:

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix
```

In [ ]:
import json
import os
import pprint

from nanohubsubmit import NanoHUBSubmitClient, SubmitRequest
from nanohubsubmit.client import CommandExecutionError, AuthenticationError
from nanohubsubmit.utils import (
    load_available_catalog,
    load_available_tools,
    load_available_venues,
    load_available_managers,
    explore_submit_server,
)

pp = pprint.PrettyPrinter(indent=2, width=120)
print("Imports loaded")

In [ ]:
# Client uses /etc/submit/submit-client.conf by default.
client = NanoHUBSubmitClient(verbose=False)
print("Client created with default config path")

## 2) Doctor Check

Validates config, auth readiness, and server capabilities.

In [ ]:
doctor = client.doctor(probe_server=True).to_dict()
print(json.dumps(doctor, indent=2, sort_keys=True))

## 3) Catalog Queries (Tools/Venues/Managers)

In [ ]:
catalog = load_available_catalog(client, operation_timeout=60.0)
print("tools:", len(catalog.tools))
print("venues:", len(catalog.venues))
print("managers:", len(catalog.managers))
print()
print("first 10 tools:")
pp.pprint(catalog.tools[:10])

In [ ]:
# Individual list helpers
tools = load_available_tools(client, operation_timeout=60.0)
venues = load_available_venues(client, operation_timeout=60.0)
managers = load_available_managers(client, operation_timeout=60.0)

print("tools sample:", tools[:5])
print("venues:", venues)
print("managers:", managers)

## 4) Full Server Exploration

In [ ]:
exploration = explore_submit_server(client, operation_timeout=60.0).to_dict()
print("doctor:")
pp.pprint(exploration["doctor"])
print()
print("venue_status entries:", len(exploration["venue_status"]))

## 5) Raw Calls (Advanced)

Use this if you want direct protocol-compatible submit arguments.

In [ ]:
raw_tools = client.raw(["--help", "tools"], operation_timeout=60.0)
print("returncode:", raw_tools.returncode)
print(raw_tools.stdout[:1200])

## 6) Validate a Submit Request Before Running

In [ ]:
request = SubmitRequest(
    command="python3",
    command_arguments=["run.py", "--size", "100"],
    venues=["workspace"],
    n_cpus=4,
    wait=True,
)
validation = client.validate_submit_request(request).to_dict()
print(json.dumps(validation, indent=2))

## 7) Submit Local Command (`ls`)

In [ ]:
local_result = client.submit(
    SubmitRequest(command="ls", local=True),
    operation_timeout=120.0,
)
print("returncode:", local_result.returncode)
print(local_result.stdout[:2000])

## 8) Remote Submit Example (Optional)

Uncomment and adapt only when you intend to launch a remote run.

In [ ]:
# remote_request = SubmitRequest(
#     command="python3",
#     command_arguments=["run.py"],
#     venues=["workspace"],
#     wait=False,
# )
# remote_result = client.submit(remote_request, operation_timeout=300.0)
# print(remote_result.to_dict())

## 9) Status / Kill / Venue Status Examples

In [ ]:
# Status for known job IDs
# status_result = client.status([12345], operation_timeout=60.0)
# print(status_result.stdout)

# Kill known job IDs (careful)
# kill_result = client.kill([12345], operation_timeout=60.0)
# print(kill_result.stdout)

venue_result = client.venue_status(operation_timeout=60.0)
print(venue_result.stdout[:1200])

## 10) Session Introspection

Scan local directories for session artifacts and optionally probe live job status.

In [ ]:
sessions = client.list_sessions(root=".", max_depth=4, include_live_status=True, limit=20).to_dict()
print("sessions discovered:", len(sessions["sessions"]))
print("inferred job ids:", sessions["inferred_job_ids"][:20])
if sessions["live_probe"] is not None:
    print("live probe ok:", sessions["live_probe"]["ok"])

## 11) Error Handling Pattern

In [ ]:
try:
    _ = load_available_catalog(client, operation_timeout=1.0)
except AuthenticationError as exc:
    print("Authentication error:", exc)
except CommandExecutionError as exc:
    print("Command execution error:", exc)
except Exception as exc:
    print("Unexpected error:", exc)

## 12) Interactive UI (`ipywidgets<8`)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

ipywidgets_major = int(widgets.__version__.split(".", 1)[0])
print("ipywidgets version:", widgets.__version__)
if ipywidgets_major >= 8:
    print("Warning: this notebook is tuned for ipywidgets<8")

In [ ]:
action = widgets.Dropdown(
    options=[
        "doctor",
        "catalog",
        "explore",
        "venue-status",
        "raw-help-tools",
        "raw-help-venues",
        "raw-help-managers",
        "submit-local",
        "status",
        "kill",
    ],
    value="doctor",
    description="Action",
)

job_ids = widgets.Text(value="", description="Job IDs", placeholder="e.g. 12345,12346")
command = widgets.Text(value="ls", description="Command")
command_args = widgets.Text(value="", description="Cmd Args", placeholder="arg1 arg2")
timeout = widgets.FloatText(value=60.0, description="Timeout")
verbose = widgets.Checkbox(value=False, description="Verbose")
run_button = widgets.Button(description="Run", button_style="primary")
output = widgets.Output()

def _parse_job_ids(text):
    values = []
    for item in text.split(","):
        item = item.strip()
        if not item:
            continue
        values.append(int(item))
    return values

def _on_run(_):
    output.clear_output()
    with output:
        try:
            c = NanoHUBSubmitClient(verbose=bool(verbose.value))
            to = float(timeout.value)

            if action.value == "doctor":
                print(json.dumps(c.doctor(probe_server=True).to_dict(), indent=2, sort_keys=True))
            elif action.value == "catalog":
                print(json.dumps(load_available_catalog(c, operation_timeout=to).to_dict(), indent=2, sort_keys=True))
            elif action.value == "explore":
                print(json.dumps(explore_submit_server(c, operation_timeout=to).to_dict(), indent=2, sort_keys=True))
            elif action.value == "venue-status":
                result = c.venue_status(operation_timeout=to)
                print(result.stdout or result.stderr)
            elif action.value == "raw-help-tools":
                result = c.raw(["--help", "tools"], operation_timeout=to)
                print(result.stdout or result.stderr)
            elif action.value == "raw-help-venues":
                result = c.raw(["--help", "venues"], operation_timeout=to)
                print(result.stdout or result.stderr)
            elif action.value == "raw-help-managers":
                result = c.raw(["--help", "managers"], operation_timeout=to)
                print(result.stdout or result.stderr)
            elif action.value == "submit-local":
                args = [x for x in command_args.value.split() if x]
                req = SubmitRequest(command=command.value.strip(), command_arguments=args, local=True)
                result = c.submit(req, operation_timeout=to)
                print(result.stdout or result.stderr)
                print("returncode:", result.returncode)
            elif action.value == "status":
                ids = _parse_job_ids(job_ids.value)
                result = c.status(ids, operation_timeout=to)
                print(result.stdout or result.stderr)
            elif action.value == "kill":
                ids = _parse_job_ids(job_ids.value)
                result = c.kill(ids, operation_timeout=to)
                print(result.stdout or result.stderr)
            else:
                print("Unknown action:", action.value)
        except Exception as exc:
            print(type(exc).__name__ + ":", exc)

run_button.on_click(_on_run)
display(widgets.VBox([action, job_ids, command, command_args, timeout, verbose, run_button, output]))

## 13) Advanced Client Overrides

Use this when you need explicit URI/timeouts instead of config defaults.

In [ ]:
advanced_client = NanoHUBSubmitClient(
    listen_uris=["tcp://128.211.145.71:830"],
    connect_timeout=10.0,
    idle_timeout=1.0,
    keepalive_interval=15.0,
    operation_timeout=60.0,
    verbose=False,
)
print("Advanced client configured")